In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformer_lens import HookedTransformer, HookedTransformerConfig, utils

In [ ]:
# model parameter constants
N_LAYERS = 1
N_HEADS = 1
D_MODEL = 32
D_HEAD = 32
D_MLP = None
D_VOCAB = 13
SEED = 123
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
MAX_DIGITS = 5
NUM_DIGITS = 2
# 10 is between numbers
CTX_LEN = NUM_DIGITS * MAX_DIGITS + 1